<a href="https://colab.research.google.com/github/bibinkunjumon2020/llm/blob/main/RAG_Zephyr_cpu_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> # **RAG - zephyr-7b-alpha with Quantization**::::



## This Experiment must need GPU. I am using the colab free credit T4 GPU. Here
## Quantization of zephyr model is performed using NF4.Model is not saved.Just Loaded directly into the llm.
## Using the full data.
## Chunking of the text after loading the pdf documents,Node parsing,Embedding using the Huginface model hkunlp/instructor-large .
## Experiment is completely free with in the free credit of Colab since the Quantization performed.

https://akash-mathur.medium.com/advanced-rag-optimizing-retrieval-with-additional-context-metadata-using-llamaindex-aeaa32d7aa2f

In [5]:
!pip install -q llama-index bitsandbytes accelerate langchain InstructorEmbedding sentence_transformers pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 4.7 MB/s eta 0:00:00


In [6]:
!ls ./data

children-in-need-of-care.pdf
civil-imprisonment.pdf
commissioner-of-oath.pdf
commissioning-of-oaths.pdf
court-annex-mediation.pdf
court-orders-judgments-rulings.pdf
default-judgements.pdf
deputy-sheriffs-court-bailiffs-messengers-of-court.pdf
deputy-sheriffs-currently-practicing.pdf
distribution-of-judgements.pdf
division-of-property.pdf
exportation-of-vehicles-to-sarpcco-countries.pdf
exportation-of-vehicles-to-the-rest-of-the-world.pdf
hierarchy-of-the-courts.pdf
interlocutory-applications.pdf
letters-to-prisoners.pdf
liquidation-judicial-management-and-sequestration-of-estates.pdf
masters-office.pdf
police-clearance.pdf
police-incident-report.pdf
pre-hearing-conference.pdf
provision-of-legal-assistance.pdf
provision-of-redress-against-administrative-decisions-of-public-functionaries.pdf
public-meetings-and-processions-permit.pdf
radiological-safety-inspections.pdf
record-of-proceedingstranscript.pdf
registration-of-home-made-trailers.pdf
registration-of-new-vehicles-purchased-from-t

In [3]:
#This is for a single File Storage
# from llama_index import download_loader,Path,Document

# PDFReader = download_loader("PDFReader")
# loader = PDFReader()
# docs = loader.load_data(file=Path("./snake-capture-services.pdf"))

# # combine all the text
# doc_text = "\n\n".join([d.get_content() for d in docs])
# documents = [Document(text=doc_text)]

## Reading the documents from the directory

In [10]:
from llama_index import SimpleDirectoryReader,Document

loader =  SimpleDirectoryReader(input_dir="./data/")
docs = loader.load_data()
print(f"Number of documents loaded = {len(docs)}")

Number of documents loaded = 95


In [11]:
print(docs[0],"\n\n\n",docs[0].metadata,"\n\n\n",docs[0].metadata["file_name"])

Doc ID: 5faeb705-e92e-44ce-971d-4253098ef006
Text: {'url': 'https://www.gov.bw/legal/access-legal-services-
government-ministriesdepartments', 'department': 'Law, Crime and
Justice', 'section': 'Legal', 'title': 'Access to Legal Services By
Government Ministries/Departments', 'contents': "Access to Legal
Services By Government Ministries/Departments\n\nOverview\nProvide
legal advice to the Govern... 


 {'page_label': '1', 'file_name': 'access-to-legal-services-by-government-ministriesdepartments.pdf', 'file_path': 'data/access-to-legal-services-by-government-ministriesdepartments.pdf', 'file_type': 'application/pdf', 'file_size': 2717, 'creation_date': '2024-01-15', 'last_modified_date': '2024-01-15', 'last_accessed_date': '2024-01-15'} 


 access-to-legal-services-by-government-ministriesdepartments.pdf


In [12]:
docs[0].text

'{\'url\':\n\'https://www.gov.bw/legal/access-legal-services-government-ministriesdepartments\',\n\'department\': \'Law, Crime and Justice\', \'section\': \'Legal\', \'title\': \'Access to\nLegal Services By Government Ministries/Departments\', \'contents\': "Access\nto Legal Services By Government\nMinistries/Departments\\n\\nOverview\\nProvide legal advice to the\nGovernment in the public interest.\\nWho is eligible?\\nGovernment Ministries\nand Departments\\nHow do I get this service?\\nConsult the\nseconded/resident Legal Advisor in the respective Ministry/Department or\ndirectly seek assistance from the office of the Attorney General.\\nHow long\nwill this service take?\\nDirectorate of Public Prosecution (DPP)Time taken to\nconclude criminal cases= 24 monthsCivil Litigation Division (CLD)Time taken\nto conclude civil cases =18 months\\xa0Legal advice given within 10\nday.Legislative Drafting Division \\xa0 (LDD)Time taken for the drafting\ncomplex bills= 14.5 months.Time taken fo

In [13]:
for document in docs:
  print(document.metadata["file_name"])

access-to-legal-services-by-government-ministriesdepartments.pdf
adjudication-of-civil-cases.pdf
adjudication-of-civil-cases.pdf
adjudication-of-matrimonial-cases.pdf
adjudication-of-matrimonial-cases.pdf
administration-of-estates-of-deceased-persons.pdf
administration-of-estates-of-deceased-persons.pdf
adoption-of-children-family-related-cases.pdf
appealed-cases-from-the-high-court-and-industrial-court.pdf
appeals-from-district-labour-office.pdf
appeals-to-land-tribunal.pdf
appeals-to-land-tribunal.pdf
application-for-arm-licence-renewable.pdf
application-for-arms-and-ammunition-dealers-licence-residents-21-years-or-above-renewable.pdf
application-for-change-of-ownership.pdf
application-for-exemption-certificate.pdf
application-for-export-permit-firearms.pdf
application-for-permit-to-acquire-firearm-change-of-ownership.pdf
application-for-permit-to-acquire-firearm-raffle.pdf
application-for-permit-to-export-firearms.pdf
application-for-permit-to-import-ammunition-resident.pdf
applicat

# *Iterating through each document text and joining it and converting the into a single llama_index Document.*

In [14]:
# combine all the text
doc_text = "\n".join([d.text for d in docs])
documents = [Document(text=doc_text)]
print(documents)

[Document(id_='08d73cc5-f520-4a8f-9681-7e52406ebb94', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='23893f00ed0a98aecf17244e09ac71a7446615ec47e9d2ef315184840694f134', text='{\'url\':\n\'https://www.gov.bw/legal/access-legal-services-government-ministriesdepartments\',\n\'department\': \'Law, Crime and Justice\', \'section\': \'Legal\', \'title\': \'Access to\nLegal Services By Government Ministries/Departments\', \'contents\': "Access\nto Legal Services By Government\nMinistries/Departments\\n\\nOverview\\nProvide legal advice to the\nGovernment in the public interest.\\nWho is eligible?\\nGovernment Ministries\nand Departments\\nHow do I get this service?\\nConsult the\nseconded/resident Legal Advisor in the respective Ministry/Department or\ndirectly seek assistance from the office of the Attorney General.\\nHow long\nwill this service take?\\nDirectorate of Public Prosecution (DPP)Time taken to\nconclude criminal

## **Chunking & Node Parsing**

In [15]:
from llama_index.text_splitter import SentenceSplitter

node_parser = SentenceSplitter(chunk_size=512)

base_nodes = node_parser.get_nodes_from_documents(documents)
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

# print all the node ids corrosponding to all the chunks
for node in base_nodes:
  print(node.id_)

node-0
node-1
node-2
node-3
node-4
node-5
node-6
node-7
node-8
node-9
node-10
node-11
node-12
node-13
node-14
node-15
node-16
node-17
node-18
node-19
node-20
node-21
node-22
node-23
node-24
node-25
node-26
node-27
node-28
node-29
node-30
node-31
node-32
node-33
node-34
node-35
node-36
node-37
node-38
node-39
node-40
node-41
node-42
node-43
node-44
node-45
node-46
node-47
node-48
node-49
node-50
node-51
node-52
node-53
node-54
node-55
node-56
node-57
node-58
node-59
node-60
node-61
node-62
node-63
node-64
node-65
node-66
node-67
node-68
node-69
node-70
node-71
node-72
node-73
node-74
node-75
node-76
node-77
node-78
node-79
node-80
node-81
node-82
node-83
node-84
node-85
node-86
node-87
node-88
node-89
node-90
node-91
node-92
node-93
node-94
node-95
node-96
node-97
node-98
node-99
node-100
node-101
node-102
node-103
node-104
node-105
node-106
node-107
node-108
node-109
node-110
node-111
node-112
node-113
node-114
node-115
node-116
node-117
node-118
node-119
node-120
node-121
node-122
nod

LLM & Embed

In [16]:
from google.colab import userdata
from transformers import BitsAndBytesConfig
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

from llama_index import PromptTemplate

## Quantization using BitsAndBytesConfig and defining a function for message template

In [17]:


# huggingface api token for downloading zephyr-7b
hf_token = userdata.get('hf_token')

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt

In [18]:
# LLM
llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-alpha",
    tokenizer_name="HuggingFaceH4/zephyr-7b-alpha",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [19]:
from langchain_community.embeddings.huggingface import HuggingFaceInstructEmbeddings

from InstructorEmbedding import INSTRUCTOR
from llama_index import ServiceContext

# embed_model = INSTRUCTOR('hkunlp/instructor-large')
# Embedding
embed_model = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large"
)

# set your ServiceContext for all the next steps
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
    )

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [23]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' #Here only cpu is allowed

model_name = "hkunlp/instructor-large"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [24]:
# set your ServiceContext for all the next steps
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=hf
    )

# Retrieving  & Querieing

## Base Retriever- It simply fetches the top-k raw text nodes by embedding similarity.


*   Creating vector index of the documents.
*   Using it to create a retriever
*   asking a prompt and returning the corresponding nodes



In [26]:
# from llama_index import VectorStoreIndex
# from llama_index.response.notebook_utils import display_source_node
# # index
# base_index = VectorStoreIndex(base_nodes, service_context=service_context)
# # find top 2 nodes
# base_retriever = base_index.as_retriever(similarity_top_k=2)

# retrievals = base_retriever.retrieve(
#     "how can we handle snake in house?"
# )

# for n in retrievals:
#     display_source_node(n, source_length=1500)

In [ ]:
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index.vector_stores import SimpleVectorStore

from llama_index import VectorStoreIndex,StorageContext
from llama_index.response.notebook_utils import display_source_node
# from llama_index.node_parser import SentenceSplitter

# create parser and parse document into nodes
# parser = SentenceSplitter()
# nodes = parser.get_nodes_from_documents(documents)

# create storage context using default stores
storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore(),
    vector_store=SimpleVectorStore(),
    index_store=SimpleIndexStore(),
)

# create (or load) docstore and add nodes
storage_context.docstore.add_documents(base_nodes)

# build index
base_index = VectorStoreIndex(base_nodes, storage_context=storage_context,service_context=service_context)

# save index
base_index.storage_context.persist(persist_dir="index_store")

# to load index later, make sure you setup the storage context
# this will loaded the persisted stores from persist_dir
loaded_storage_context = StorageContext.from_defaults(persist_dir="index_store")



In [ ]:
# then load the index object
from llama_index import load_index_from_storage

loaded_index = load_index_from_storage(loaded_storage_context)

In [30]:

# find top 2 nodes
base_retriever = base_index.as_retriever(similarity_top_k=2)

retrievals = base_retriever.retrieve(
    "how can we handle snake in house?"
)

for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-158<br>**Similarity:** 0.8327450577491173<br>**Text:** (+267)
3956381\xa0FrancistownPrivate Bag F19FrancistownBotswanaTel. (+267)
2413793", 'documents': ['Letter of Demand', 'Withdrawal', 'Writ of Execution
and Ejectment', 'Summons']}

{'url': 'https://www.gov.bw/public-safety/snake-capture-services',
'department': 'Law, Crime and Justice', 'section': 'Public Safety', 'title': 'Snake
Capture Services', 'contents': "Snake Capture
Services\n\nOverview\nBotswana Defence Force (BDF) provides snake
capture services, in Gaborone and its periphery. With the escalating reports
of snakes found in homesteads the BDF has since engaged in assisting
members of the community in capturing such dangerous reptiles.Caution:Do
not attempt to capture the snakes by yourselfIf you are not in danger do not
attempt to kill the snakeIf there is a snake bite:Do not panic but remain
calmDo not cut the wound or attempt to remove the venomImmobilise the
bitten part and stay as quiet as possible to keep the poison from spreading
through your bodyRemove jewelry before you start to swellApply a splint to
reduce blood movement of the affected area but keep it loose enough so as
not to restrict blood flowDon't drink caffeine and alcoholCleanse the wound
but don't flush it with water, cover it with a clean dry clothPosition yourself if
possible so that the bite is at or below the level of your heartDon't try to
capture the snake but try to remember its type,colour and shape so that you
can describe it, which will help in your treatment.Call 998 or seek immediat...<br>

**Node ID:** node-157<br>**Similarity:** 0.8088508578617342<br>**Text:** (+267)
3956381\xa0FrancistownPrivate Bag F19FrancistownBotswanaTel. (+267)
2413793", 'documents': ['Letter of Demand', 'Withdrawal', 'Writ of Execution
and Ejectment', 'Summons']}

{'url': 'https://www.gov.bw/public-safety/snake-capture-services',
'department': 'Law, Crime and Justice', 'section': 'Public Safety', 'title': 'Snake
Capture Services', 'contents': "Snake Capture
Services\n\nOverview\nBotswana Defence Force (BDF) provides snake
capture services, in Gaborone and its periphery.<br>

In [31]:
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine
# create a query engine
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)


In [32]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [33]:
%%time
# query
response = query_engine_base.query(
    "oh my god there is a snake?"
)
print(response)

If you encounter a snake and need assistance, you can contact the Snake Handlers at Sir Seretse Khama Barracks in Mogoditshane, Botswana. The service is provided 24 hours a day, 7 days a week, and is available to members of the public in and around Gaborone. The cost for this service is P100.00. For more information, you can contact the Directorate of Protocol and Public Affairs at the Botswana Defence Force.
CPU times: user 13.3 s, sys: 717 ms, total: 14 s
Wall time: 18.1 s


In [35]:
%%time
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

# find top 2 nodes
base_retriever = base_index.as_retriever(similarity_top_k=2)
# create a query engine
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

# query
response = query_engine_base.query(
    "snake bite me?"
)
print(response)


If you have been bitten by a snake, the provided service can assist you. The Botswana Defence Force (BDF) provides snake capture services in Gaborone and its periphery. In case of a snake bite, it is recommended to remain calm, not cut the wound, and not attempt to remove the venom. Immobilize the bitten part, stay as quiet as possible, and seek immediate medical attention. The service is provided 24 hours a day, 7 days a week, and costs P100.00. The Snake Handlers can be contacted at Sir Seretse Khama Barracks, Mogoditshane, or by phone at 3662060 or 75669120.
CPU times: user 14 s, sys: 330 ms, total: 14.3 s
Wall time: 14.3 s


In [37]:
%%time
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

# find top 2 nodes
base_retriever = base_index.as_retriever(similarity_top_k=2)
# create a query engine
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

# query
response = query_engine_base.query(
    "who can write letter to prisoners"
)
print(response)


Family members and friends of prisoners are permitted to communicate with prisoners through letters.
CPU times: user 2.55 s, sys: 394 ms, total: 2.95 s
Wall time: 2.95 s


In [38]:
%%time
from llama_index.query_engine.retriever_query_engine import RetrieverQueryEngine

# find top 2 nodes
base_retriever = base_index.as_retriever(similarity_top_k=2)
# create a query engine
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

# query
response = query_engine_base.query(
    "i need a police clearance?"
)
print(response)

If you need a police clearance, you can obtain it for either changing the vehicle color or registering a pre-owned vehicle purchased from the rest of the world. To get this service, you need to bring the vehicle for inspection at a designated police establishment. The supporting documents required for both services are identification documents, registration book (blue book), and a letter from the dealer who painted the vehicle for changing the vehicle color, and identification documents, purchase invoice, manufacturer's certificate, and bill of lading for registering a pre-owned vehicle purchased from the rest of the world. The service is free, and it takes one working day to complete. You can obtain this service from the Serious Crime Squad at Plot 356, Extension 4, Independence Avenue, Gaborone.
CPU times: user 14.6 s, sys: 434 ms, total: 15 s
Wall time: 15.1 s
